# 키워드 검색 - Lexical Search


## 사전 준비


### 필요한 패키지 설치


In [3]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q boto3
!pip install pandas

  Using cached pandas-2.2.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (114 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp311-cp311-macosx_11_0_arm64.whl (11.3 MB)
Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


### 데이터 준비


In [2]:
import pandas as pd
import requests

df = pd.read_csv("../data/movies.csv", low_memory=False)
df.head(5)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url
0,1917,드라마|전쟁,2020,2.19,8.88,5017,제1차 세계대전이 한창인 1917년. 독일군에 의해 모든 통신망이 파괴된 상황 속에...,조지 맥케이|딘-찰스 채프먼,콜린 퍼스|베네딕트 컴버배치|마크 스트롱|앤드류 스캇|리차드 매든|에드리언 스카보로...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200212_161/1...
1,21 브릿지: 테러 셧다운,범죄|드라마|스릴러,2020,1.01,7.84,579,"뉴욕 맨해튼 중심에서 벌어진 경찰 연쇄 살해 사건, 범인을 잡기 위해 베테랑 경찰 ...",채드윅 보스만|J.K. 시몬스|시에나 밀러|테일러 키취,스테판 제임스|빅토리아 카타게나|키스 데이빗|샤이나 라이언|알렉산더 시디그|모로코 ...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191227_205/1...
2,n번째 이별중,코미디|멜로/로맨스,2020,4.01,6.48,177,여자친구 ‘데비’에게 대차게 차여 충격받은 물리학 천재 ‘스틸먼’. 사랑하는 그녀의...,에이사 버터필드|소피 터너|스카일러 거손도,오브리 레이놀즈|윌 펠츠|질리안 조이|케이든 J. 그레고리,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200409_56/15...
3,가슴 큰 태희,멜로/로맨스,2020,2.27,8.67,3,곧 결혼을 앞둔 예비 신혼 부부 차욱과 민주. 차욱은 직장 후배인 용우네 부부에게 ...,정보없음,정보없음,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200211_88/15...
4,개인교습,멜로/로맨스,2020,3.02,10.00,1,부잣집 프랑스 아가씨 마고에게 첫눈에 반한 발리의 가난한 어부 에카는 그녀에게 피아...,하리 산티카|도르카스 카핀,정보없음,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200226_62/15...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           2100 non-null   int64  
 1   title        2100 non-null   object 
 2   author       2100 non-null   object 
 3   genre        2100 non-null   object 
 4   description  2100 non-null   object 
 5   rating       2100 non-null   float64
 6   date         2100 non-null   object 
 7   completed    2100 non-null   bool   
 8   age          1998 non-null   object 
 9   free         2100 non-null   bool   
 10  link         2100 non-null   object 
dtypes: bool(2), float64(1), int64(1), object(7)
memory usage: 151.9+ KB


## 인덱스 생성 및 데이터 인제스트


### OpenSearch 클라이언트 생성


In [3]:
# Add system path for utils
import sys

sys.path.insert(0, "/Users/jinhwan/Repository/Labs/opensearch-with-bedrock-workshop-kr/utils")

In [4]:
import boto3, json
from utils import get_cfn_outputs

region_name = "us-east-1"

cfn = boto3.client("cloudformation", region_name)
kms = boto3.client("secretsmanager", region_name)

stackname = "opensearch-workshop"
cfn_outputs = get_cfn_outputs(stackname, cfn)

aos_credentials = json.loads(
    kms.get_secret_value(SecretId=cfn_outputs["OpenSearchSecret"])["SecretString"]
)

aos_host = cfn_outputs["OpenSearchDomainEndpoint"]

### OpenSearch 클러스터에 연결


In [5]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

auth = (aos_credentials["username"], aos_credentials["password"])

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

nori analyzer를 사용해 연결 테스트


In [6]:
request_body = {"analyzer": "nori", "text": "OpenSearch 워크샵에 오신 고객 여러분 환영합니다."}

# Send the request to the _analyze endpoint
response = aos_client.indices.analyze(body=request_body)

# Print the response
print(json.dumps(response, indent=4, ensure_ascii=False))

{
    "tokens": [
        {
            "token": "opensearch",
            "start_offset": 0,
            "end_offset": 10,
            "type": "word",
            "position": 0
        },
        {
            "token": "워크",
            "start_offset": 11,
            "end_offset": 13,
            "type": "word",
            "position": 1
        },
        {
            "token": "샵",
            "start_offset": 13,
            "end_offset": 14,
            "type": "word",
            "position": 2
        },
        {
            "token": "오",
            "start_offset": 16,
            "end_offset": 17,
            "type": "word",
            "position": 4
        },
        {
            "token": "고객",
            "start_offset": 19,
            "end_offset": 21,
            "type": "word",
            "position": 7
        },
        {
            "token": "여러분",
            "start_offset": 22,
            "end_offset": 25,
            "type": "word",
            "position": 8
   

### 인덱스 생성


In [7]:
index_name = "movie_lexical"

movie_lexical = {
    "settings": {
        "number_of_replicas": 0,
        "number_of_shards": 1,
        "max_result_window": 15000,
        "analysis": {"analyzer": {"analysis-nori": {"type": "nori", "stopwords": "_korean_"}}},
    },
    "mappings": {
        "properties": {
            "date": {
                "type": "float",
            },
            "genre": {
                "type": "text",
            },
            "img_url": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "main_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "page_url": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "text": {
                "type": "text",
            },
            "rating": {"type": "float"},
            "supp_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "title": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "vote_count": {"type": "long"},
            "year": {"type": "long"},
        }
    },
}

In [8]:
aos_client.indices.create(index=index_name, body=movie_lexical)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'movie_lexical'}

In [9]:
aos_client.indices.get(index=index_name)

{'movie_lexical': {'aliases': {},
  'mappings': {'properties': {'date': {'type': 'float'},
    'genre': {'type': 'text'},
    'img_url': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'main_act': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'page_url': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'rating': {'type': 'float'},
    'supp_act': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'text': {'type': 'text'},
    'title': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'vote_count': {'type': 'long'},
    'year': {'type': 'long'}}},
  'settings': {'index': {'replication': {'type': 'DOCUMENT'},
    'number_of_shards': '1',
    'provided_name': 'movie_lexical',
    'max_result_window': '15000',
    'creation_date': '1717250735165',
    'analysis': {'analyzer': {

### 데이터 인제스트


In [10]:
from tqdm import tqdm
from opensearchpy import helpers

json_data = df.to_json(orient="records", lines=True)
docs = json_data.split("\n")[:-1]  # To remove the last empty line


def _generate_data():
    for doc in docs:
        yield {"_index": index_name, "_source": doc}


succeeded = []
failed = []
for success, item in helpers.parallel_bulk(
    aos_client, actions=_generate_data(), chunk_size=10, thread_count=2, queue_size=2
):
    if success:
        succeeded.append(item)
    else:
        failed.append(item)

## 키워드 검색


In [120]:
def keyword_search(query_text):
    query = {
        "size": 10,
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["text"],
            }
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_id"],
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["text"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_id", "_score", "title", "plot", "genre", "rating"]
    )
    display(query_result_df)

In [124]:
keyword_search("강호 제일검이 어린 아이로 다시 태어난다")

,_score,id,title,author,genre,description,date,completed,age,free,link
0,10.995780,750184,나쁜사람,둠스,"에피소드, 액션",착한 남자가 좋다는 그녀의 이상형이 되기 위해 10년 동안 노력했다. 그런데! 갑자...,2022.12.27 22:58,False,12세 이용가,False,https://comic.naver.com/webtoon/list?titleId=7...
1,7.779238,786910,레지나레나 - 용서받지 못...,곽나나 / 설동원 / 김영지,"스토리, 로맨스",“기회를 드릴게요. 제게 용서받을 기회요.” 아버지는 딸을 팔았다. 그리고 딸은 지...,2022.10.23 22:52,True,전체연령가,False,https://comic.naver.com/webtoon/list?titleId=7...
2,7.173797,773793,필리아로제 - 가시왕관의 ...,백화등 / Ryuta / 김영지,"스토리, 판타지","소녀 사제, 패륜 왕자의 반란을 막아라!상대의 마음을 읽는 능력을 가진 사제 필리아...",2022.12.26 22:54,False,전체연령가,False,https://comic.naver.com/webtoon/list?titleId=7...
3,6.378121,796152,마루는 강쥐,모죠,"에피소드, 개그","우리 집 강아지 마루가 사람이 되었다, 그것도 5살 아이로!강아지 + 어린아이의 무...",2022.12.26 22:51,False,전체연령가,False,https://comic.naver.com/webtoon/list?titleId=7...
4,6.282778,777853,다름이 아니라,해미,"스토리, 드라마","어린 시절, 반 아이들에게 '다름'의 초능력을 자랑하려다 거짓말쟁이로 몰리게 된 '...",2022.06.02 22:59,True,전체연령가,True,https://comic.naver.com/webtoon/list?titleId=7...
5,6.099657,804318,죽음으로 구원하사,고요빛 / 유니,"스토리, 스릴러",연쇄살인마에게 친구들을 잃고 혼자 살아남은 '이설희'는 불길한 아이로 찍혀 괴롭힘을...,2022.12.29 22:00,False,18세 이용가,False,https://comic.naver.com/webtoon/list?titleId=8...
6,6.057405,726467,틴맘,theterm,"스토리, 드라마",나 혼자서 잘 할수 있을까?어린 엄마의 좌충우돌 육아일기,2021.03.12 23:14,True,12세 이용가,True,https://comic.naver.com/webtoon/list?titleId=7...
7,5.813714,183558,"스마일 브러시, 오래된 사...",와루,"에피소드, 일상",오래된 사진에서 찾아보는 어린 시절의 이야기.\r\n당신의 앨범에서 잃어버렸던 추억...,2011.07.29 00:10,True,전체연령가,False,https://comic.naver.com/webtoon/list?titleId=1...
8,5.588872,25695,향수,석우,"스토리, 스릴러","행복했었던 나의 어린 시절. 그러나, 그에게는 잔인한 기억일지도 모른다. 향수, 그...",2008.11.25 12:13,True,전체연령가,False,https://comic.naver.com/webtoon/list?titleId=2...
9,5.430841,777221,다시 또 봄,이힝,"스토리, 감성",미움받는 게 싫어서 거절을 잘 못하는 소심한 봄이는 할머니에게 고민을 털어놓으며 스...,2022.07.16 22:57,True,전체연령가,True,https://comic.naver.com/webtoon/list?titleId=7...


# Clean Up


In [ ]:
aos_client.indices.delete(index=index_name)